<a href="https://colab.research.google.com/github/amritanshkm/CITS4403-Project/blob/main/01_Insolvency_and_Wealth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#AFSA Insolvency and Wealth Data

Using the AFSA data, we will analyse the relationship the relationship between insolvency/bankruptcy and wealth distribution. This process includes:

1. Data Preparation
2. Exploratory Data Analysis
3. Generating visualisations, outputs, and observations


#Data Preparation

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
#personal insolvency
#regional insolvency

pinsol = '/content/drive/MyDrive/CITS4403/quarterly_personal_insolvencies.csv'
rinsol = '/content/drive/MyDrive/CITS4403/regional_quarterly_time_series.csv'

In [47]:
import pandas as pd

pinsolpd = pd.read_csv(pinsol)
rinsolpd = pd.read_csv(rinsol)

In [48]:
pinsolpd

,Quarter,Type of personal insolvency administration,State,In a business or company,Number of people entering a new personal insolvency,Unnamed: 5,Unnamed: 6
0,Sep-07,Debtor's petition,New South Wales,Yes,641,NaN,NaN
1,Sep-07,Debtor's petition,Victoria,Yes,377,NaN,NaN
2,Sep-07,Debtor's petition,Queensland,Yes,367,NaN,NaN
3,Sep-07,Debtor's petition,South Australia,Yes,96,NaN,NaN
4,Sep-07,Debtor's petition,Western Australia,Yes,81,NaN,NaN
...,...,...,...,...,...,...,...
15115,Jun-25,Total personal insolvencies,Northern Territory,Total,11,NaN,NaN
15116,Jun-25,Total personal insolvencies,Western Australia,Total,236,NaN,NaN
15117,Jun-25,Total personal insolvencies,Tasmania,Total,79,NaN,NaN
15118,Jun-25,Total personal insolvencies,Other,Total,229,NaN,NaN


In [49]:
#checking NA values

pinsolpd.info()

rinsolpd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120 entries, 0 to 15119
Data columns (total 7 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Quarter                                              15120 non-null  object 
 1   Type of personal insolvency administration           15120 non-null  object 
 2   State                                                15120 non-null  object 
 3   In a business or company                             15120 non-null  object 
 4   Number of people entering a new personal insolvency  15120 non-null  int64  
 5   Unnamed: 5                                           0 non-null      float64
 6   Unnamed: 6                                           0 non-null      float64
dtypes: float64(2), int64(1), object(4)
memory usage: 827.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78624 entries, 0 to 78623
Data columns (

Pinsol has two empty columns due to the structure of the csv file. Deleting them.

There are otherwise no NA values, but many columns are incorrect data types.

Additionally, quarters are

In [50]:
pinsolpd = pinsolpd.drop(['Unnamed: 5', 'Unnamed: 6'], axis = 1)

pinsolpd['Quarter'] = pd.to_datetime(pinsolpd['Quarter'], format='%b-%y').dt.to_period('Q')

pinsolpd['Type of personal insolvency administration'] = (
    pinsolpd['Type of personal insolvency administration'].astype('category')
)

pinsolpd['State'] = (
    pinsolpd['State'].astype('category')
)

pinsolpd['In a business or company'] = (
    pinsolpd['In a business or company'].astype('category')
)

pinsolpd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15120 entries, 0 to 15119
Data columns (total 5 columns):
 #   Column                                               Non-Null Count  Dtype        
---  ------                                               --------------  -----        
 0   Quarter                                              15120 non-null  period[Q-DEC]
 1   Type of personal insolvency administration           15120 non-null  category     
 2   State                                                15120 non-null  category     
 3   In a business or company                             15120 non-null  category     
 4   Number of people entering a new personal insolvency  15120 non-null  int64        
dtypes: category(3), int64(1), period[Q-DEC](1)
memory usage: 281.5 KB


In [51]:
rinsolpd['Quarter'] = pd.to_datetime(rinsolpd['Quarter'], format='%b-%y').dt.to_period('Q')


rinsolpd['State'] = (
    rinsolpd['State'].astype('category')
)

rinsolpd['In a business or company'] = (
    rinsolpd['In a business or company'].astype('category')
)

rinsolpd['Number of people entering a new personal insolvency'] = pd.to_numeric(
    rinsolpd['Number of people entering a new personal insolvency'],
    errors='coerce'
)

rinsolpd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78624 entries, 0 to 78623
Data columns (total 6 columns):
 #   Column                                               Non-Null Count  Dtype        
---  ------                                               --------------  -----        
 0   Quarter                                              78624 non-null  period[Q-DEC]
 1   State                                                78624 non-null  category     
 2   ASGS Code                                            78624 non-null  object       
 3   ASGS Name                                            78624 non-null  object       
 4   In a business or company                             78624 non-null  category     
 5   Number of people entering a new personal insolvency  60823 non-null  float64      
dtypes: category(2), float64(1), object(2), period[Q-DEC](1)
memory usage: 2.5+ MB


Wealth inequality datas

In [81]:
w1 = '/content/drive/MyDrive/CITS4403/CSV/Wealth/table3.1.1.csv'
w2 = '/content/drive/MyDrive/CITS4403/CSV/Wealth/table3.1.2.csv'

In [82]:
w1df = pd.read_csv(w1)
w2df = pd.read_csv(w2)

w1df.info()
w2df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Gross household income decile  11 non-null     object 
 1   Lowest                         11 non-null     float64
 2   Second                         11 non-null     float64
 3   Third                          11 non-null     float64
 4   Fourth                         11 non-null     float64
 5   Fifth                          11 non-null     float64
 6   Sixth                          11 non-null     float64
 7   Seventh                        11 non-null     float64
 8   Eighth                         11 non-null     float64
 9   Ninth                          11 non-null     float64
 10  Highest                        11 non-null     float64
 11  All households                 11 non-null     float64
 12  Unnamed: 12                    0 non-null      float

In [83]:
w1df = w1df.drop(['Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'], axis = 1)
w1df['All households'] = pd.to_numeric(
    w1df['All households'],
    errors='coerce'
)
w1df.info()


w2df = w2df.drop(['Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15'], axis = 1)
w2df['Lowest'] = pd.to_numeric(
    w2df['Lowest'],
    errors='coerce'
)
w2df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Gross household income decile  11 non-null     object 
 1   Lowest                         11 non-null     float64
 2   Second                         11 non-null     float64
 3   Third                          11 non-null     float64
 4   Fourth                         11 non-null     float64
 5   Fifth                          11 non-null     float64
 6   Sixth                          11 non-null     float64
 7   Seventh                        11 non-null     float64
 8   Eighth                         11 non-null     float64
 9   Ninth                          11 non-null     float64
 10  Highest                        11 non-null     float64
 11  All households                 11 non-null     float64
dtypes: float64(11), object(1)
memory usage: 1.2+ KB
<cla

In [84]:
w1df

,Gross household income decile,Lowest,Second,Third,Fourth,Fifth,Sixth,Seventh,Eighth,Ninth,Highest,All households
0,Lowest,201.2,90.7,68.7,120.4,124.2,106.2,92.9,55.3,65.4,49.8,971.9
1,Second,215.2,116.2,81.2,90.0,113.4,102.5,98.0,62.5,61.6,28.3,974.4
2,Third,143.5,128.3,83.3,85.3,102.3,123.1,113.6,94.1,63.2,42.6,973.6
3,Fourth,132.0,120.8,118.0,68.2,85.6,87.7,82.9,101.8,101.6,63.6,971.4
4,Fifth,94.3,140.7,148.3,91.7,86.1,92.6,79.9,92.4,80.0,72.9,979.0
5,Sixth,75.2,128.9,126.3,116.7,109.8,94.0,86.2,97.9,71.4,68.7,967.8
6,Seventh,56.2,113.5,123.3,116.3,98.0,100.6,92.1,96.3,81.1,98.9,972.5
7,Eighth,37.3,83.8,116.2,124.6,109.0,86.9,114.2,107.4,99.6,100.0,971.1
8,Ninth,17.2,36.3,79.1,100.3,97.0,110.4,105.9,125.9,163.0,137.6,976.2
9,Highest,6.7,14.9,29.3,51.7,50.3,62.5,112.8,130.4,196.4,316.8,974.1


In [85]:
w2df

,Gross household income decile,Lowest,Second,Third,Fourth,Fifth,Sixth,Seventh,Eighth,Ninth,Highest,All households
0,Lowest,20.7,9.3,7.1,12.4,12.7,11.0,9.5,5.7,6.7,5.1,100.0
1,Second,22.2,11.9,8.3,9.3,11.6,10.6,10.0,6.5,6.3,2.9,100.0
2,Third,14.8,13.1,8.6,8.8,10.5,12.7,11.6,9.7,6.5,4.4,100.0
3,Fourth,13.6,12.4,12.1,7.0,8.8,9.1,8.5,10.5,10.4,6.5,100.0
4,Fifth,9.7,14.4,15.2,9.4,8.8,9.6,8.2,9.5,8.2,7.5,100.0
5,Sixth,7.7,13.2,13.0,12.0,11.2,9.7,8.8,10.1,7.3,7.1,100.0
6,Seventh,5.8,11.6,12.7,12.0,10.0,10.4,9.4,9.9,8.3,10.2,100.0
7,Eighth,3.8,8.6,11.9,12.8,11.2,9.0,11.7,11.1,10.2,10.3,100.0
8,Ninth,1.8,3.7,8.1,10.3,9.9,11.4,10.9,13.0,16.7,14.1,100.0
9,Highest,0.7,1.5,3.0,5.3,5.2,6.5,11.6,13.5,20.1,32.6,100.0


#Exploratory Data Analysis

#Outputs

#Observations and Summary